**This notebook covered by the following [license](License.ipynb)  This note must not be removed**

See [Configure Service Account](https://kubernetes.io/docs/tasks/configure-pod-container/configure-service-account/)

In [1]:
kubectl get serviceaccount default -o yaml

apiVersion: v1
kind: ServiceAccount
metadata:
  creationTimestamp: "2021-11-30T09:04:57Z"
  name: default
  namespace: default
  resourceVersion: "428"
  uid: 8fee6829-8d24-4dd1-8596-b807318e6ebe
secrets:
- name: default-token-4wm25


In [2]:
kubectl patch serviceaccount default  --patch "automountServiceAccountToken: false"

serviceaccount/default patched


In [3]:
kubectl get serviceaccount default -o yaml | grep -E '(^|automountServiceAccountToken.*)'

apiVersion: v1
automountServiceAccountToken: false
kind: ServiceAccount
metadata:
  creationTimestamp: "2021-11-30T09:04:57Z"
  name: default
  namespace: default
  resourceVersion: "13482"
  uid: 8fee6829-8d24-4dd1-8596-b807318e6ebe
secrets:
- name: default-token-4wm25


# ☠️ **this can be overwritten** ☠️



In [4]:
kubectl apply -f - <<EOF
apiVersion: apps/v1
kind: Deployment
metadata:
  name: nginx-with-token-again
  labels:
    app: nginx-with-token-again
spec:
  replicas: 1
  selector:
    matchLabels:
      app: nginx-with-token-again
  template:
    metadata:
      labels:
        app: nginx-with-token-again
    spec:
      automountServiceAccountToken: true
      containers:
      - name: nginx
        image: nginxinc/nginx-unprivileged
        ports:
        - containerPort: 8080
EOF

deployment.apps/nginx-with-token-again configured


In [12]:
kubectl get pods
POD=$(kubectl get pods -o jsonpath='{.items[0].metadata.name}' )
echo $POD

NAME                                     READY   STATUS    RESTARTS   AGE
nginx-strict-5cdc48cf55-5ckph            1/1     Running   0          24m
nginx-strict-5cdc48cf55-mnwbq            1/1     Running   0          24m
nginx-strict-5cdc48cf55-s645d            1/1     Running   0          24m
nginx-with-token-again-7d7cd958c-7fjxd   1/1     Running   0          20s
nginx-strict-5cdc48cf55-5ckph


In [13]:
kubectl exec  nginx-with-token-again-7d7cd958c-7fjxd -- df

Filesystem                    1K-blocks     Used Available Use% Mounted on
overlay                        61796348 12550976  46083260  22% /
tmpfs                             65536        0     65536   0% /dev
tmpfs                           7888556        0   7888556   0% /sys/fs/cgroup
/dev/mapper/sirius--vg-docker  61796348 12550976  46083260  22% /etc/hosts
shm                               65536        0     65536   0% /dev/shm
tmpfs                           7888556        0   7888556   0% /proc/acpi
tmpfs                           7888556        0   7888556   0% /proc/scsi
tmpfs                           7888556        0   7888556   0% /sys/firmware


In [9]:
kubectl get pods  -o yaml | grep -i token

        - serviceAccountToken:
            path: token
        - serviceAccountToken:
            path: token
        - serviceAccountToken:
            path: token
    generateName: nginx-with-token-again-66868cff8b-
      app: nginx-with-token-again
    name: nginx-with-token-again-66868cff8b-pzttq
      name: nginx-with-token-again-66868cff8b
    automountServiceAccountToken: true
        - serviceAccountToken:
            path: token


In [10]:
kubectl get pods -A -o jsonpath='{.items[*].spec.automountServiceAccountToken}'

true